## Gemini Pro 기본 동작 예제

Feedback : shins777@gmail.com. 

* 이 Colab은 Gemini Pro 의 기본 동작을 설명하기 위한 예제입니다.  
* 코드는 Langchain을 기반으로 처리합니다.  
    * 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

### 라이브러리 설치

In [ ]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai

In [ ]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [ ]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
    

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.

In [ ]:
model_name="gemini-pro"
project="ai-hangsik"
location="asia-northeast3"

### Responsible AI

구글은 아래와 같이 생성형 AI 처리시 Responsible AI 부분으로 Harmcategory 에 따른 LLM 응답 조절이 가능합니다.  
구글은 구글의 모델을 사용하는 사용자가 생성된 컨텐츠때문에 피해가 되지 않도록 하기 위해서 Responsible AI라는 내용으로 구글의 모델을 사용하는 사용자를 보고하고 있습니다.
이 부분은 특별한 문제가 없다면 생략해도 됩니다.

Responsible AI 설정을 위한 클래스입니다.  
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold

In [ ]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

### 모델 초기화 및 기본 실행

아래 내용은 Langchain 기반에서의 gemini_pro 를 초기화하는 방법입니다.  
업무에 따라서 Langchain 을 사용하셔도 좋고, 만일 latency 가 중요한 업무이면 Google native API를 사용하는것도 Latency를 줄이는 방법이 될수 있습니다.

* Langchain VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI
* API 참고(Python SDK) : https://cloud.google.com/vertex-ai/generative-ai/docs/sdk-for-llm/llm-sdk-overview
* API 참고(REST API) : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/overview?hl=ko

Vertex AI에 대한 Langchain 라이브러리를 보시려면 아래 URL을 참고하세요.
* https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm

참고!! : Langchain 에서 Google AI, GoogleGenerativeAI 또는 langchain-google-genai 로 표시된 API는 Consumer 버전의 API이므로 Enteprise(기업)용 사용자는 반드시 위의 Vertex AI 용으로 사용해주세요.

In [ ]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name=model_name,
                  project=project,
                  location=location,
                  verbose=True,
                  streaming=True,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 10
                 )

아래 URL 에 제시된 Langchain 라이브러리를 활용하여 다양한 처리를 구현해보실수 있습니다.
* Langchain VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI


In [ ]:
from langchain.prompts import PromptTemplate

context ="""
광복 후 70년을 맞은 2015년 한국은 전환기를 맞고 있다. 고령화사회(65세 이상 인구가 14% 이상)로 다가가고 있는 우리나라는 2060년이 되면 고령인구 비중이 40.1%가 돼 세계에서 두 번째로 늙은 나라가 될 것이라는 전망이 나온다. 출산율은 1.23명(2010∼2014년 평균)으로 세계에서 네 번째로 낮다. 그만큼 경제활동을 할 수 있는 인구가 적어진다는 뜻이다.
한국 경제가 이제 저성장 국면으로 접어든 것이 아니냐는 우려도 나온다. 1961년부터 50년간 한국의 연간 경제성장률이 3%를 밑돈 것은 다섯 차례로 오일쇼크, 외환위기, 카드사태, 글로벌 금융위기 등의 특수한 이유가 있었다. 그마저도 충격을 딛고 1~2년 안에 경제가 빠른 반등에 성공했다. 하지만 2012년부터는 이렇다 할 외부충격이 없는데도 작년(3.3%)을 제외하고는 계속 2%대의 낮은 성장세를 보이고 있다.
‘제2의 도약’을 위해서는 한계점에 다다른 기존 산업의 성장, 비정규직·정규직으로 분화된 노동시장, 세대갈등, 청년실업 등 풀어야 할 문제가 한둘이 아니다. 외환위기 직후 재정경제부 수장으로 경제정책을 이끈 강봉균 전 장관은 “한국은 과감한 개방 등 다른 개발도상국이 못하던 것들을 해내 선진국 문턱으로 올라설 수 있었다”며 “선진국들이 힘들어하는 구조개혁에 성공하면 다시 한번 도약할 수 있을 것”이라고 말했다."""

aspect = "문제점"

prompt = PromptTemplate.from_template("아래 Context 에 대해서 {aspect} 관점에서 요약 해주세요. Context : {context} ")
prompt = prompt.format(aspect=aspect,
                       context=context)

response = gemini_pro.invoke(prompt)
display(Markdown(response))


### LLMChain 기본 실행
아래는 문장에서 Fact 를 요약하고, 다른 컨텍스트는 요약을 하는 처리 예입니다. 
PromptTemplate 은 Langchain 을 사용할때 효과적으로 Prompting 을 할수 있수 있습니다. 아래 내용 참고하세요.
*   PromptTemplate: https://api.python.langchain.com/en/stable/langchain_api_reference.html#module-langchain.prompts

In [ ]:
from langchain.chains import LLMChain

context1 ="""
광복 후 70년을 맞은 2015년 한국은 전환기를 맞고 있다. 고령화사회(65세 이상 인구가 14% 이상)로 다가가고 있는 우리나라는 2060년이 되면 고령인구 비중이 40.1%가 돼 세계에서 두 번째로 늙은 나라가 될 것이라는 전망이 나온다. 출산율은 1.23명(2010∼2014년 평균)으로 세계에서 네 번째로 낮다. 그만큼 경제활동을 할 수 있는 인구가 적어진다는 뜻이다.
한국 경제가 이제 저성장 국면으로 접어든 것이 아니냐는 우려도 나온다. 1961년부터 50년간 한국의 연간 경제성장률이 3%를 밑돈 것은 다섯 차례로 오일쇼크, 외환위기, 카드사태, 글로벌 금융위기 등의 특수한 이유가 있었다. 그마저도 충격을 딛고 1~2년 안에 경제가 빠른 반등에 성공했다. 하지만 2012년부터는 이렇다 할 외부충격이 없는데도 작년(3.3%)을 제외하고는 계속 2%대의 낮은 성장세를 보이고 있다.
‘제2의 도약’을 위해서는 한계점에 다다른 기존 산업의 성장, 비정규직·정규직으로 분화된 노동시장, 세대갈등, 청년실업 등 풀어야 할 문제가 한둘이 아니다. 외환위기 직후 재정경제부 수장으로 경제정책을 이끈 강봉균 전 장관은 “한국은 과감한 개방 등 다른 개발도상국이 못하던 것들을 해내 선진국 문턱으로 올라설 수 있었다”며 “선진국들이 힘들어하는 구조개혁에 성공하면 다시 한번 도약할 수 있을 것”이라고 말했다."""

context2 = """
한국 헌법은 “모든 국민의 재산권은 보장된다”라고 명시하고 있다. 시장 경제 체제를 기본으로 하 고, 개인과 기업의 자유로운 경제 활동을 존중하며, 그로써 얻은 개인과 기업의 이득과 재산을 모 두 보장한다.
그러나 헌법은 자본주의적 시장 경제를 무제한 보장하지 않는다. 헌법은 자본을 지나치게 남용해 타인에게 피해를 주면 이를 조정할 수 있도록 규정하고 있다. 이는 자유 시장 경제의 문제점을 보완 하기 위한 최소한의 장치이다."""

prompt = PromptTemplate(
    input_variables=["text_input1", "text_input2"],
    template="""이 텍스트에서 핵심 사실을 추출해 보세요. 의견을 포함하지 마세요. 각 사실에 대해서 상세하게 설명해주세요.  : {text_input1}
    그리고 아래 context: 부분을 요약해주세요. context : {text_input2}
    """
)

fact_extraction_chain = LLMChain(llm=gemini_pro, prompt=prompt, verbose=True)
response = fact_extraction_chain.run(text_input1=context1,text_input2=context2 )

display(Markdown(response))


### SequentialChain 예제
* 두개의 연관된 질문을 한번의 호출로 연계.
* 이전 답변에서 사실을 추출해서 다음 질문에 활용할 때 사용.
* LLMChain : https://api.python.langchain.com/en/stable/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain



In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["country"],
    template="{country}의 수도는 어디인가요 ?",
)
capital_chain = LLMChain(llm=gemini_pro, prompt=prompt_1, verbose=True, output_key="capital")

prompt_2 = PromptTemplate(
    input_variables=["capital"],
    template=" {capital} 에서 가장 유명한 곳 10 군데를 알려주세요.",
)
places_chain = LLMChain(llm=gemini_pro, prompt=prompt_2, verbose=True, output_key="places")


In [ ]:
from langchain.chains import SequentialChain

# https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html#

overall_chain : SequentialChain = SequentialChain(
      chains=[capital_chain, places_chain],
      input_variables=["country"], # 이곳에 처음 입력 받는 변수를 정의.
      output_variables=["capital", "places"], # 이곳에 여러개 return 되는 변수를 정의.
      verbose=True
    )

output_from_ai : dict = overall_chain({"country":"대한민국"})

for key, value in output_from_ai.items():
  display(Markdown(value))
